In [1]:
from sentinelhub import (
    SHConfig,
    BBox,
    bbox_to_dimensions,
    DataCollection,
    MimeType,
    MosaickingOrder,
    SentinelHubRequest,
    BBoxSplitter
)

import folium
import math
from tqdm import tqdm

In [2]:
def get_image(coordinates):
    
    # put here your sentinel hub credential
    config = SHConfig()
    config.sh_client_id = "1d1d404c-fa78-46e9-9620-09b2e5371237"
    config.sh_client_secret = "hh9l0BqjjPaeFk7Ga2sotrNXeLsgeVIx"

    # customize this functions to specify the band you want to download
    evalscript = """
        // VERSION=3
        
        function setup() {
            return {
                input: [{
                    bands: [
                        "B08",
                        "B11"
                    ],
                }],
                output: {
                    bands: 2,
                }
            };
        }

        function evaluatePixel(sample) {
            return [
                sample.B08,
                sample.B11
            ];
        }
    """

    # put here the coordinate of the image you want to download
    # coordinates = (-96.455, 41.146, -96.450, 41.153)

    # set the coordinate reference system
    crs = 4326

    # set the desired resolution
    resolution = 10

    # set the time interval
    time_interval = ("2023-08-01", "2023-08-31")

    # set the source of the image
    data_collection = DataCollection.SENTINEL2_L2A

    # set the constant class for type of mosaicking order
    mosaicking_order = MosaickingOrder.LEAST_CC

    # update this value if you want to save the image
    save_data = True

    # set the folder where you want to dowload the image
    data_folder = './images'

    # set the format of the image
    response_format = MimeType.TIFF

    bbox = BBox(bbox = coordinates, crs = crs)
    size = bbox_to_dimensions(bbox, resolution = resolution)
    request = SentinelHubRequest(
        config = config,
        bbox = bbox,
        size = size,
        evalscript = evalscript,
        data_folder = data_folder,
        input_data = [
            SentinelHubRequest.input_data(
                time_interval = time_interval,
                data_collection = data_collection,
                mosaicking_order = mosaicking_order
            )
        ],
        responses = [
            SentinelHubRequest.output_response(
                identifier = "default",
                response_format = response_format
            )
        ]
    )

    return request.get_data(save_data=save_data)

In [3]:
def divide_bbox(bbox:BBox, resolution:float, max_size=512):
    dimensions = bbox_to_dimensions(bbox, resolution)
    if dimensions[0] > max_size or dimensions[1] > max_size:
        x_divisions = max(1, math.ceil(dimensions[0] / max_size))
        y_divisions = max(1, math.ceil(dimensions[1] / max_size))
        bbox_splitter = BBoxSplitter([bbox], 4326, (x_divisions, y_divisions))
        return bbox_splitter.get_bbox_list()
    else:
        return [bbox]

left = 6.7499552751 # italia
bottom = 36.619987291
right = 18.4802470232
top = 47.1153931748

map = folium.Map()

for bbox in tqdm(divide_bbox(bbox=BBox((left, bottom, right, top), 4326), resolution=30)):
    left, bottom, right, top = bbox
    folium.Rectangle([(top, left), (bottom, right)], color='black', weight=1).add_to(map)
    # get_image(bbox)
    # break

map

100%|██████████| 4950/4950 [00:00<00:00, 78614.61it/s]
